
<br>
Assignment 5: CRNN For Text Recognition<br>
Course Coordinator: Dr. Manojkumar Ramteke<br>
Teaching Assistant: Abdur Rahman<br>
This code is for educational purposes only. Unauthorized copying or distribution without the consent of the course coordinator is prohibited.<br>
Copyright © 2024. All rights reserved.<br>


In [ ]:
import torch
class ConverterForCTC(object):
    """ Convert between text-label and text-index """
    def __init__(self, character):
        # character (str): set of the possible characters.
        dict_character = list(character)
        self.dict = {}
        for i, char in enumerate(dict_character):
            # NOTE: 0 is reserved for 'CTCblank' token required by CTCLoss
            self.dict[char] = i + 1
        self.character = ['[CTCblank]'] + dict_character  # dummy '[CTCblank]' token for CTCLoss (index 0)
    def encode(self, text, batch_max_length=25):
        """convert text-label into text-index.
        input:
            text: text labels of each image. [batch_size]
            batch_max_length: max length of text label in the batch. 25 by default
        output:
            text: text index for CTCLoss. [batch_size, batch_max_length]
            length: length of each text. [batch_size]
        """
        length = [len(s) for s in text]

        # The index used for padding (=0) would not affect the CTC loss calculation.
        batch_text = torch.LongTensor(len(text), batch_max_length).fill_(0)
        for i, t in enumerate(text):
            text = list(t)
            text = [self.dict[char] for char in text]
            batch_text[i][:len(text)] = torch.LongTensor(text)
        return (batch_text, torch.IntTensor(length))
    def decode(self, text_index, length):
        """ convert text-index into text-label. """
        texts = []
        for index, l in enumerate(length):
            t = text_index[index, :]
            char_list = []
            for i in range(l):
                if t[i] != 0 and (not (i > 0 and t[i - 1] == t[i])):  # removing repeated characters and blank.
                    char_list.append(self.character[t[i]])
            text = ''.join(char_list)
            texts.append(text)
        return texts
    
########## Example usage ##########
# character = '0123456789' # [CTCblank] is automatically added at the beginning
# converter = ConverterForCTC(character)
# text = ['123', '456'] # Batch Size = 2

encoded_text, length = converter.encode(text, batch_max_length=5)<br>
print(encoded_text) # tensor([[2, 3, 4, 0, 0], [5, 6, 7, 0, 0]])<br>
print(length) # tensor([3, 3])

decoded_text = converter.decode(encoded_text, length)<br>
print(decoded_text) # ['123', '456']